In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import datetime as dt

In [2]:
from tqdm import tqdm_notebook as tqdm

In [4]:
path_to_result = "/storage/score_prediction_KM/2021-01-26_docking_results_to_tables"
# path_to_result = '/home/margaritakovaleva/score_prediction/2021-01-26_docking_results_to_tables/'

# Choose between 'Morgan' and 'AtomPairs'
fingerprints = "Morgan"

# Whether to cut or no to cut positive scores
positive_scores = False

number_of_rows = 1_000_000


In [5]:
import os

if fingerprints == "Morgan":
    directory = "prediction_results_Morgan"
elif fingerprints == "AtomPairs":
    directory = "prediction_results_AP"

if postive_score == True:
    directrory += "_uncut"

if os.path.exists(f"{os.getcwd()}/{directory}/"):
    pass
else:
    os.mkdir(f"{os.getcwd()}/{directory}/")


In [6]:
def loggg(with_dataframe=True):
    def decorator(f):
        def wrapper(dataf, *args, **kwargs):
            tic = dt.datetime.now()  # Время начала выполнения функции
            result = f(dataf, *args, **kwargs)  # Выполнения функции
            toc = dt.datetime.now()  # Время окончания работы функции

            if with_dataframe == True:
                # Если у dataf и result есть аттрибут shape, то добавляются названия колонн и делается заптсь об изменении shape
                if hasattr(dataf, "shape") and hasattr(result, "shape"):
                    share_before = dataf.shape
                    shape_after = result.shape
                    added_columns = set(result.columns) - set(dataf.columns)
                    print(
                        f"{f.__name__},  shape {dataf.shape}->{result.shape},  took={toc-tic}"
                    )

            # запись о том, сколько времени занял этап
            else:
                print(f"{f.__name__} took={toc-tic}")

            return result

        return wrapper

    return decorator


In [7]:
@loggg()
def load_csv_with_dtypes(filename, sep=","):
    with open(filename) as fin:
        for first_line in fin:
            break
        column_names = first_line.split(sep)
    dtype = {
        col.strip(): "bool" if col.startswith("fps") else "string"
        for col in column_names
        if col.strip()
    }
    dtype["Score"] = "float"

    df = pd.read_csv(filename, dtype=dtype, index_col=0)
    return df


@loggg()
def load_data_from_glob_with_dtypes(glob_expression, n_files=2):
    import glob

    rv = []
    for csv_filename in tqdm(glob.glob(glob_expression)[:n_files]):
        df = load_csv_with_dtypes(csv_filename)

        # # assert we read only booleans and strings
        # helper_types = pd.DataFrame({'a':['string'], 'b':[True]}).convert_dtypes().dtypes
        # assert set(df.dtypes.values) == set(helper_types), csv_filename

        rv.append(df)
    df = pd.concat(rv).reset_index()

    return df


# @loggg()
# def load_data(pdb):
#     full_path = path_to_result + pdb + '/' + file_with_result

#     types_dict = {'Name':'string', 'ZincID': 'string'}
#     for i in range(2048):
#         types_dict[str(i)] = 'bool'

#     table = pd.read_csv(full_path,
#                         header = 0,
#                         sep = '\t',
#                         index_col = 0,
#                         dtype = types_dict,
#                         nrows = number_of_rows)
#     return table


@loggg()
def start_pipeline(dataf):
    return dataf.copy()


@loggg()
def convert_score_as_float(dataf):
    dataf["Score"] = dataf.Score.astype("float")
    return dataf


@loggg()
def add_label(table):
    # for percentage in (0.01, 0.05, 0.5):
    for percentage in (0.005, 0.01, 0.02, 0.05):
        name = f"Hit(top {percentage*100}%)"
        pos = round(percentage * table.shape[0])
        threshold = table.sort_values("Score").Score[pos]

        table[name] = list(map(lambda x: int(x < threshold), table["Score"]))
        # np.where(table.Score < threshold, 1, 0)
    return table


@loggg()
def delete_high_scores(table):
    if positive_scores == False:
        table["Score"] = np.where(table["Score"] > 0, 0, table["Score"])
    return table


In [8]:
# assuming that path_to result contains only directories(named with pdbs) with docking results
import os

pdbs = os.listdir(path=path_to_result)


In [9]:
% % time

raw_data = {}

for pdb in pdbs:
    print('PDBid: ', pdb)
    if fingerprints == 'Morgan':
        glob = f"{path_to_result}/{pdb}/*binarised.csv"
    elif fingerprints == 'AtomPairs':
        glob = f"{path_to_result}/{pdb}/*binarised_AP.csv"
    raw_data[pdb] = (
        load_data_from_glob_with_dtypes(glob, n_files=100)
        .pipe(delete_high_scores)
    )


PDBid:  4eiy


<ipython-input-7-041d767e0df8>:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for csv_filename in tqdm(glob.glob(glob_expression)[:n_files]):



delete_high_scores,  shape (969760, 2052)->(969760, 2052),  took=0:00:00.028417
CPU times: user 8min 26s, sys: 6.55 s, total: 8min 33s
Wall time: 8min 38s


In [10]:
pdb = "4eiy"


In [11]:
% % time

df = (
    raw_data[pdb]
    .pipe(start_pipeline)
    .set_index('ZincID')
    .drop(['Smiles', 'index'], axis=1)
    .pipe(add_label)
)


start_pipeline,  shape (969760, 2052)->(969760, 2052),  took=0:00:02.117851
add_label,  shape (969760, 2053)->(969760, 2053),  took=0:00:22.250132
CPU times: user 22.2 s, sys: 3.36 s, total: 25.5 s
Wall time: 25.6 s


In [12]:
df = add_label(df)

add_label,  shape (969760, 2053)->(969760, 2053),  took=0:00:21.187701


In [13]:
df

,Score,fps_bin__type=Morgan__size=2048_0,fps_bin__type=Morgan__size=2048_1,fps_bin__type=Morgan__size=2048_2,fps_bin__type=Morgan__size=2048_3,fps_bin__type=Morgan__size=2048_4,fps_bin__type=Morgan__size=2048_5,fps_bin__type=Morgan__size=2048_6,fps_bin__type=Morgan__size=2048_7,fps_bin__type=Morgan__size=2048_8,...,fps_bin__type=Morgan__size=2048_2042,fps_bin__type=Morgan__size=2048_2043,fps_bin__type=Morgan__size=2048_2044,fps_bin__type=Morgan__size=2048_2045,fps_bin__type=Morgan__size=2048_2046,fps_bin__type=Morgan__size=2048_2047,Hit(top 0.5%),Hit(top 1.0%),Hit(top 2.0%),Hit(top 5.0%)
ZincID,,,,,,,,,,,,,,,,,,,,,
ZINC000985611501,-13.424853,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0,0,0,0
ZINC000985567531,-1.623259,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,0,0,0,0
ZINC000985727430,-13.722397,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0,0,0,0
ZINC000985886800,-5.732475,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0,0,0,0
ZINC000985663916,-14.858214,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZINC000060640402,-20.783788,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0,0,0,0
ZINC000597676169,-12.185357,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0,0,0,0
ZINC000597755533,-3.525244,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,0,0,0,0


In [14]:
import collections

hit_cols = [col for col in df.columns if col.startswith("Hit")]


for hit in hit_cols:
    y = df[hit].values
    print(hit, collections.Counter(y))


Hit(top 0.5%) Counter({0: 964911, 1: 4849})
Hit(top 1.0%) Counter({0: 960062, 1: 9698})
Hit(top 2.0%) Counter({0: 950365, 1: 19395})
Hit(top 5.0%) Counter({0: 921272, 1: 48488})


# Additional models

In [179]:
from sklearn.base import BaseEstimator, RegressorMixin


class RandomGaussianRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, mean: float = -10, std: float = 10):
        self._mean = mean
        self._std = std

        def RandomGaussianSampler(arr):
            return np.random.normal(loc=mean, scale=std, size=arr.shape[0])

        self._model = RandomGaussianSampler

    def fit(self, X, y):
        pass

    def predict(self, X):
        y = self._model(X)
        return y

    def fit_predict(self, X, y):
        self.fit(X, y)
        return self.predict(X)


In [180]:
from sklearn.base import BaseEstimator, RegressorMixin


class RandomUniformRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, low: float = -50, high: float = 0):
        self._low = low
        self._high = high

        def RandomUniformSampler(arr):
            return np.random.uniform(low=low, high=high, size=arr.shape[0])

        self._model = RandomUniformSampler

    def fit(self, X, y):
        pass

    def predict(self, X):
        y = self._model(X)
        return y

    def fit_predict(self, X, y):
        self.fit(X, y)


# Parallelization

# let's define model zoo

In [222]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC

temp_class_models = [
    KNeighborsClassifier,
    #                     DecisionTreeClassifier,
    RandomForestClassifier,
    #                     DummyClassifier,
    #                     SVC,
    #                     SGDClassifier,
]

class_models = {}

for model in temp_class_models:
    if "Classifier" in model.__name__:
        class_models[model.__name__] = model()
    else:
        class_models[model.__name__ + "Classifier"] = model()

# class_models['MostFrequent' + DummyClassifier.__name__] = DummyClassifier(strategy='most_frequent')
class_models[KNeighborsClassifier.__name__ + "WithOneNeighbor"] = KNeighborsClassifier(
    n_neighbors=1
)


In [223]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR, LinearSVR

temp_regr_models = [
    #                    LinearRegression,
    #                    Ridge,
    #                    Lasso,
    KNeighborsRegressor,
    #                    DecisionTreeRegressor,
    RandomForestRegressor,
    #                    RandomGaussianRegressor,
    #                    RandomUniformRegressor,
    SVR,
    LinearSVR,
]

regr_models = {}

for model in temp_regr_models:
    regr_models[model.__name__] = model()

regr_models[KNeighborsRegressor.__name__ + "WithOneNeighbor"] = KNeighborsRegressor(
    n_neighbors=1
)


## let's prepare folds in advance

In [182]:
import multiprocessing

hit_cols = [col for col in df.columns if col.startswith("Hit")]


manager = multiprocessing.Manager()
ns = manager.Namespace()

ns.X = df.drop(["Score"], axis=1).drop(hit_cols, axis=1)
ns.y = df[hit_cols]
ns.score = df["Score"]


In [183]:
# hit_cols = [col for col in df.columns if col.startswith('Hit')]

# X = df.drop(['Score'], axis = 1).drop(hit_cols, axis = 1)
# y = df[hit_cols]

In [184]:
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import StratifiedKFold
% % time


n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

# sample_sizes = [10_000, 100_000, 500_000]
# sample_sizes = [200_000, 400_000]
sample_sizes = [10_000, 50_000, 100_000, 200_000, 400_000]

folds = {}
for sample_size in tqdm(sample_sizes, desc='sample_sizes'):
    print(f'Sample_size = {sample_size}')
    source_df = df.sample(n=sample_size, random_state=0)

    for col in tqdm(hit_cols, desc='hit_cols'):

        X_temp = (source_df
                  .drop(hit_cols, axis=1)
                  .drop('Score', axis=1)
                  .values
                  )

        y_temp = source_df[col].values

        train_zinc, test_zinc = [], []
        for fold_idx, (train_index, test_index) in enumerate(skf.split(X_temp, y_temp)):
            fold_id = (pdb, sample_size, col, fold_idx)
            train_zinc = np.array([source_df.index[i] for i in train_index])
            test_zinc = np.array([source_df.index[i] for i in test_index])
            folds[fold_id] = (train_zinc, test_zinc)


<timed exec>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Sample_size = 10000


<timed exec>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Sample_size = 50000


<timed exec>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...


Sample_size = 200000


<timed exec>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Sample_size = 400000


<timed exec>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`




CPU times: user 1min 53s, sys: 38.4 s, total: 2min 31s
Wall time: 2min 31s


### Folds for classification

In [224]:
from itertools import product

class_params = {}
for (fold_key, fold), (model_key, model) in product(
    folds.items(), class_models.items()
):
    class_params[fold_key, model_key] = (fold, model)


### Folds for regression

In [225]:
from itertools import product

regr_params = {}
for (fold_key, fold), (model_key, model) in product(folds.items(), regr_models.items()):
    regr_params[fold_key, model_key] = (fold, model)

In [187]:
def timing(func):
    from functools import wraps

    @wraps
    def inner(*args, **kwargs):
        import datetime as dt

        tic = dt.datetime.now()
        rv = func(*args, **kwargs)
        toc = dt.datetime.now()

        print(f"    Function {func.__name__} took {toc - tic}")
        return rv

    return inner


In [188]:
# def do_prediction(parameters, X, y):
def do_prediction(parameters, namespace, directory):
    import gc

    gc.collect()

    import datetime as dt

    tic = dt.datetime.now()

    # inner function start

    (fold_key, model_key), (fold, model) = parameters
    (train_zinc, test_zinc) = fold

    (pdb, sample_size, col, fold_idx) = fold_key

    train_size = len(train_zinc)

    #     if fingerprints == 'Morgan':
    #         directory = 'prediction_results_Morgan'
    #     elif fingerprints == 'AtomPairs':
    #         directory = 'prediction_results_AP'

    filename = f"{directory}/{pdb}/model={model_key}_trainsize={train_size}_col={col}_dx={fold_idx}.json"

    # directory creation
    import os

    if os.path.exists(f"{directory}/{pdb}"):
        pass
    else:
        os.mkdir(f"{directory}/{pdb}")

    if os.path.exists(filename):
        toc = dt.datetime.now()
        print(f"{toc}:Prediction already done\n")
        return None
    else:
        Xt, Xv = namespace.X.loc[train_zinc], namespace.X.loc[test_zinc]

        if model_key.find("Classifier") == -1:
            yt, yv = (
                namespace.score[train_zinc].values,
                namespace.score[test_zinc].values,
            )
        else:
            yt, yv = (
                namespace.y[col][train_zinc].values,
                namespace.y[col][test_zinc].values,
            )
        # define model and X, y

        #         clf = model()
        clf = model
        clf.fit(Xt, yt)

        # import resource
        # print(f'Resources usage: {resource.getrusage(RUSAGE_SELF)}')

        if model_key.find("Classifier") == -1:
            proba = clf.predict(Xv)
        else:
            proba = clf.predict_proba(Xv)

        # convert types to json serializable
        proba = proba.tolist()
        yv = yv.tolist()

        toc = dt.datetime.now() - tic
        toc = [toc.days, toc.seconds, toc.microseconds]

        # write to json file
        d = {
            "prediction": proba,
            "true_labels": yv,
            "time_for_prediction": toc,
        }

        import json

        with open(filename, "w") as fin:
            json.dump(d, fin)

        # inner function stop
        toc = dt.datetime.now()

        # print(f'Resources usage: {resource.getrusage(RUSAGE_SELF)}')

        print(
            (
                f"{toc}: Finished for model {model_key}, fold {fold_key}\n"
                f"    This took {toc - tic}"
            )
        )

        del clf
        del proba
        del yv
        gc.collect()
        return filename


In [226]:
from random import sample

dataspace = list(class_params.items()) + list(regr_params.items())
# dataspace = list(class_params.items())

shuffled_dataspace = sample(dataspace, len(dataspace))
# dataspace =  list(regr_params.items())


In [227]:
from functools import partial

if fingerprints == "Morgan":
    directory = "prediction_results_Morgan"
elif fingerprints == "AtomPairs":
    directory = "prediction_results_AP"

do_prediction_with_namespace = partial(do_prediction, namespace=ns, directory=directory)


In [228]:
import multiprocessing as mp

with mp.Pool(15) as p:
    rv = p.map(do_prediction_with_namespace, dataspace)

2021-03-19 09:41:01.204276:Prediction already done

2021-03-19 09:41:01.232002:Prediction already done

2021-03-19 09:41:01.276219:Prediction already done

2021-03-19 09:41:01.306901:Prediction already done

2021-03-19 09:41:01.325260:Prediction already done

2021-03-19 09:41:01.410125:Prediction already done
2021-03-19 09:41:01.410052:Prediction already done


2021-03-19 09:41:01.480175:Prediction already done

2021-03-19 09:41:01.489644:Prediction already done

2021-03-19 09:41:01.587145:Prediction already done

2021-03-19 09:41:01.607860:Prediction already done

2021-03-19 09:41:01.775232:Prediction already done

2021-03-19 09:41:04.359969:Prediction already done
2021-03-19 09:41:04.360476:Prediction already done


2021-03-19 09:41:04.508628:Prediction already done

2021-03-19 09:41:04.636137:Prediction already done

2021-03-19 09:41:04.913913:Prediction already done

2021-03-19 09:41:06.816747:Prediction already done

2021-03-19 09:41:06.866283:Prediction already done

2021-03-19 0

    This took 0:11:57.206910
2021-03-19 10:05:18.504659:Prediction already done

2021-03-19 10:05:18.786164:Prediction already done

2021-03-19 10:06:04.683037: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4eiy', 50000, 'Hit(top 0.5%)', 2)
    This took 0:11:33.733113
2021-03-19 10:06:06.313188:Prediction already done

2021-03-19 10:13:24.355587: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4eiy', 50000, 'Hit(top 5.0%)', 4)
    This took 0:11:16.696762
2021-03-19 10:13:24.825686:Prediction already done

2021-03-19 10:13:25.055387:Prediction already done

2021-03-19 10:14:14.021732: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4eiy', 50000, 'Hit(top 2.0%)', 1)
    This took 0:11:36.230363
2021-03-19 10:14:14.515610:Prediction already done

2021-03-19 10:14:14.738877:Prediction already done

2021-03-19 10:14:15.299390:Prediction already done

2021-03-19 10:16:48.890514: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4e


2021-03-19 10:41:43.662695:Prediction already done

2021-03-19 10:41:43.966632:Prediction already done

2021-03-19 10:41:44.264948:Prediction already done

2021-03-19 10:41:44.539972:Prediction already done

2021-03-19 10:41:44.754463:Prediction already done

2021-03-19 10:41:45.033409:Prediction already done

2021-03-19 10:41:45.324886:Prediction already done

2021-03-19 10:41:45.639428:Prediction already done

2021-03-19 10:41:45.922563:Prediction already done

2021-03-19 10:41:46.182034:Prediction already done

2021-03-19 10:41:46.439320:Prediction already done

2021-03-19 10:41:46.700234:Prediction already done

2021-03-19 10:41:46.961219:Prediction already done

2021-03-19 10:41:47.220540:Prediction already done

2021-03-19 10:41:47.480975:Prediction already done

2021-03-19 10:41:47.741804:Prediction already done

2021-03-19 10:41:48.002107:Prediction already done

2021-03-19 10:41:48.261566:Prediction already done

2021-03-19 10:41:48.522749:Prediction already done

2021-03-19 


2021-03-19 10:42:29.248393:Prediction already done

2021-03-19 10:42:29.533413:Prediction already done

2021-03-19 10:42:29.820437:Prediction already done

2021-03-19 10:42:30.151609:Prediction already done

2021-03-19 10:42:30.434533:Prediction already done

2021-03-19 10:42:30.720718:Prediction already done

2021-03-19 10:42:31.002913:Prediction already done

2021-03-19 10:42:31.284969:Prediction already done

2021-03-19 10:42:31.568329:Prediction already done

2021-03-19 10:42:31.850768:Prediction already done

2021-03-19 10:42:32.132092:Prediction already done

2021-03-19 10:42:32.413978:Prediction already done

2021-03-19 10:42:32.694440:Prediction already done

2021-03-19 10:42:32.976201:Prediction already done

2021-03-19 10:42:33.258905:Prediction already done

2021-03-19 10:42:33.542158:Prediction already done

2021-03-19 10:42:33.823378:Prediction already done

2021-03-19 10:42:34.288328:Prediction already done

2021-03-19 10:42:34.667172:Prediction already done

2021-03-19 


2021-03-19 16:13:33.682861:Prediction already done

2021-03-19 17:27:24.854659: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4eiy', 200000, 'Hit(top 0.5%)', 2)
    This took 2:39:19.216574
2021-03-19 17:27:27.763139:Prediction already done

2021-03-19 17:27:56.106489: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4eiy', 200000, 'Hit(top 2.0%)', 3)
    This took 2:35:18.582837
2021-03-19 17:29:36.305656: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4eiy', 200000, 'Hit(top 5.0%)', 1)
    This took 2:30:59.704803
2021-03-19 17:29:36.854586:Prediction already done

2021-03-19 17:29:37.098468:Prediction already done

2021-03-19 17:31:18.492460: Finished for model RandomForestClassifier, fold ('4eiy', 200000, 'Hit(top 0.5%)', 3)
    This took 0:03:50.449434
2021-03-19 17:32:26.118817: Finished for model KNeighborsClassifierWithOneNeighbor, fold ('4eiy', 200000, 'Hit(top 1.0%)', 2)
    This took 2:32:10.948016
2021-03-19 17:32:27.208521:Pr

/opt/conda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-03-19 17:52:54.109392: Finished for model LinearSVR, fold ('4eiy', 200000, 'Hit(top 2.0%)', 1)
    This took 0:03:39.739093
2021-03-19 18:33:52.673604: Finished for model SVR, fold ('4eiy', 100000, 'Hit(top 5.0%)', 4)
    This took 7:07:12.159935
2021-03-19 18:34:03.506078:Prediction already done

2021-03-19 18:34:04.226685:Prediction already done

2021-03-19 18:34:04.498686:Prediction already done

2021-03-19 18:34:04.768502:Prediction already done



KeyboardInterrupt: 